In [54]:
import numpy as np
import cv2
import os
import tensorflow as tf

# Define the frame extraction function
def extract_frames(video_path, frame_dir, frame_step=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(0, frame_count, frame_step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(frame_dir, f"frame_{i}.jpg")
        cv2.imwrite(frame_path, frame)
    
    cap.release()

# Detect frame disorientation (for demonstration, consider disorientation as large brightness changes)
def detect_disorientation(frame, prev_frame, threshold=50):
    if prev_frame is None:
        return False
    # Convert frames to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    # Compute the absolute difference
    diff = cv2.absdiff(gray_frame, gray_prev_frame)
    
    # Check if the mean of the differences exceeds the threshold
    return np.mean(diff) > threshold

# Define the prediction function
def predict_video(video_path, model, frame_dir=r'C:\Users\Sandhya\Desktop\sih\frames', frame_step=5):
    # Extract frames from the video
    temp_frame_dir = os.path.join(frame_dir, "temp")
    if not os.path.exists(temp_frame_dir):
        os.makedirs(temp_frame_dir)
    
    extract_frames(video_path, temp_frame_dir)

    # Load and preprocess the frames
    frames = [cv2.imread(os.path.join(temp_frame_dir, frame)) for frame in sorted(os.listdir(temp_frame_dir))]
    frames = [cv2.resize(frame, (224, 224)) for frame in frames]
    
    # Normalize frames
    norm_frames = np.array(frames) / 255.0  

    # Detect disorientation and print disoriented frames
    prev_frame = None
    for i, frame in enumerate(frames):
        if detect_disorientation(frame, prev_frame):
            print(f"Disoriented frame: frame_{i}")
            cv2.imshow(f"Disoriented frame: {i}", frame)
            cv2.waitKey(500)  # Display frame for 500ms
        prev_frame = frame
    cv2.destroyAllWindows()

    # Make prediction
    norm_frames = np.expand_dims(norm_frames, axis=0)  # Add batch dimension
    prediction = model.predict(norm_frames)

    # Print the prediction value
    print(f"Prediction value: {prediction[0][0]}")

    # Remove temporary frames
    for f in os.listdir(temp_frame_dir):
        os.remove(os.path.join(temp_frame_dir, f))
    os.rmdir(temp_frame_dir)  # Remove the temp directory
    
    return "Fake" if prediction < 0.35 else "Real"

# Load the model
model = tf.keras.models.load_model(r'my_model.keras')

# Example usage with a test video path
video_path = r"C:\Users\Sandhya\Desktop\sih\real1.mp4"
result = predict_video(video_path, model)
print(f"Prediction: {result}")


1/1 [==============================] - 4s 4s/step
Prediction value: 0.33821922540664673
Prediction: Fake
